### Imports

In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

## Redução de linhas por modelo

## Tempo medio de cada modelo por fontes

#

## Precisão dos modelos por fontes

### Inferências

In [2]:
import pandas as pd
import json
import glob
import re
from pathlib import Path

def extract_analysis_data(analysis_str):
    # Primeiro tenta com blocos markdown
    match = re.search(r"```json\s*(\{.*?\})\s*```", analysis_str, re.DOTALL)
    if not match:
        # fallback: tenta pegar qualquer JSON com CLASSIFICATION
        match = re.search(r"\{[^{}]*\"CLASSIFICATION\"[^{}]*\}", analysis_str, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(1 if '```' in match.group(0) else 0))
        except json.JSONDecodeError:
            return None
    return None

# Encontrar todos os arquivos JSONL no padrão especificado
files = glob.glob('inference/**/*.jsonl', recursive=True)

data = []

for file_path in files:
    # Extrair o modelo do caminho do arquivo
    path = Path(file_path)
    model = path.parts[1]  # Extrai o segundo componente do caminho (inference/{model}/...)
    if(path.parts[2] == "data"):
        font = path.parts[2]
        type = path.parts[3]
    else:
        font = path.parts[3]
        type = path.parts[4]
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                record = json.loads(line)
                analysis_data = extract_analysis_data(record.get('analysis', ''))

                data.append({
                    'model': model,
                    'type': type,
                    'font': font,
                    'source_file': record.get('file', ''),
                    'classification': analysis_data.get('CLASSIFICATION', "NOT INTERESTING") if analysis_data else "NOT INTERESTING",
                    'confidence': analysis_data.get('CONFIDENCE', 0) if analysis_data else 0
                })

            except Exception as e:
                print(f"Erro ao processar linha do arquivo {file_path}: {str(e)}")

# Criar DataFrame
df = pd.DataFrame(data)

# Exibir estrutura do DataFrame
print("-----------------------------------")
print(f"Total de registros processados: {len(df)}")
print("\nPrimeiros registros:")
display(df)
df.to_csv("classifications.csv", index=False)

-----------------------------------
Total de registros processados: 5418

Primeiros registros:


,model,type,font,source_file,classification,confidence
0,deepseek-r1_14b,attack,data,data/attack/20250411_175749.jsonl,INTERESTING,0.95
1,deepseek-r1_14b,attack,data,data/attack/20250411_175749.jsonl,NOT INTERESTING,0
2,deepseek-r1_14b,attack,data,data/attack/20250411_175749.jsonl,NOT INTERESTING,0.7
3,deepseek-r1_14b,attack,data,data/attack/20250411_175806.jsonl,INTERESTING,0.85
4,deepseek-r1_14b,attack,data,data/attack/20250411_175826.jsonl,INTERESTING,0.85
...,...,...,...,...,...,...
5413,qwen3_14b,safe,tinyllama_1.1b,filtering/tinyllama_1.1b/safe/safe_20250505_19...,INTERESTING,0.65
5414,qwen3_14b,safe,tinyllama_1.1b,filtering/tinyllama_1.1b/safe/safe_20250505_19...,NOT INTERESTING,0
5415,qwen3_14b,safe,tinyllama_1.1b,filtering/tinyllama_1.1b/safe/safe_20250505_19...,NOT INTERESTING,0
5416,qwen3_14b,safe,tinyllama_1.1b,filtering/tinyllama_1.1b/safe/safe_20250505_20...,NOT INTERESTING,0.65


### Métricas

### Gerando inferências concretas

In [3]:
import pandas as pd

def process_group(group):
    if group.empty:
        return pd.DataFrame()
    
    # Extrair metadados fixos do grupo
    model = group['model'].iloc[0]
    source_file = group['source_file'].iloc[0]
    type_ = group['type'].iloc[0]  # Assume que 'type' é consistente no grupo
    font = group['font'].iloc[0]   # Assume que 'font' é consistente no grupo

    # Contar ocorrências de cada classificação
    class_counts = group['classification'].value_counts()
    max_count = class_counts.max()
    
    # Identificar classes com contagem máxima (podem ser múltiplas em caso de empate)
    top_classes = class_counts[class_counts == max_count].index.tolist()
    
    # Critério de desempate: priorizar 'INTERESTING'
    if len(top_classes) > 1 and 'INTERESTING' in top_classes:
        majority_class = 'INTERESTING'
    else:
        majority_class = class_counts.idxmax()  # Escolhe a classe mais frequente

    # Calcular métricas consolidadas
    consolidated = pd.DataFrame({
        'model': [model],
        'font': [font],
        'source_file': [source_file],
        'type': [type_],
        'classification': [majority_class],
        'confidence': [group['confidence'].mean()],  # Média das confianças
    })
    
    return consolidated

# Carregar e processar dados
df = pd.read_csv('classifications.csv')
df['confidence'] = pd.to_numeric(df['confidence'], errors='coerce')
df = df.dropna(subset=['confidence'])

# Agrupar por modelo e arquivo, processar e consolidar
processed = (
    df.groupby(['model', 'source_file'], group_keys=False)  # Agrupamento chave
    .apply(process_group)
    .reset_index(drop=True)
)

# Ordenar colunas e salvar
final_df = processed[['model', 'font', 'source_file', 'type', 'classification', 'confidence']]
final_df.to_csv('consolidated_classifications.csv', index=False)

C:\Users\lucas\AppData\Local\Temp\ipykernel_19628\1639364311.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(process_group)


### Gerando métricas finais

In [4]:
import numpy as np

final_df["false_positive"] = np.where(
    (final_df["type"] == "safe") &
    (final_df["classification"] == "INTERESTING"),
    1,
    0
)

final_df["false_negative"] = np.where(
    (final_df["type"] == "attack") &
    (final_df["classification"] == "NOT INTERESTING"),
    1,
    0
)

final_df["true_positive"] = np.where(
    (final_df["type"] == "attack") &
    (final_df["classification"] == "INTERESTING"),
    1,
    0
)

final_df["true_negative"] = np.where(
    (final_df["type"] == "safe") &
    (final_df["classification"] == "NOT INTERESTING"),
    1,
    0
)

display(final_df)
final_df.to_csv("consolidated_classifications.csv", index=False)

,model,font,source_file,type,classification,confidence,false_positive,false_negative,true_positive,true_negative
0,deepseek-r1_14b,data,data/attack/20250411_175749.jsonl,attack,NOT INTERESTING,0.550000,0,1,0,0
1,deepseek-r1_14b,data,data/attack/20250411_175806.jsonl,attack,INTERESTING,0.850000,0,0,1,0
2,deepseek-r1_14b,data,data/attack/20250411_175826.jsonl,attack,INTERESTING,0.800000,0,0,1,0
3,deepseek-r1_14b,data,data/attack/20250411_175855.jsonl,attack,INTERESTING,0.375000,0,0,1,0
4,deepseek-r1_14b,data,data/attack/20250411_175959.jsonl,attack,INTERESTING,0.516667,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
1525,qwen3_14b,tinyllama_1.1b,filtering/tinyllama_1.1b/safe/safe_20250505_19...,safe,NOT INTERESTING,0.000000,0,0,0,1
1526,qwen3_14b,tinyllama_1.1b,filtering/tinyllama_1.1b/safe/safe_20250505_19...,safe,NOT INTERESTING,0.072222,0,0,0,1
1527,qwen3_14b,tinyllama_1.1b,filtering/tinyllama_1.1b/safe/safe_20250505_19...,safe,NOT INTERESTING,0.000000,0,0,0,1
1528,qwen3_14b,tinyllama_1.1b,filtering/tinyllama_1.1b/safe/safe_20250505_20...,safe,NOT INTERESTING,0.650000,0,0,0,1


In [5]:
import pandas as pd

# Carregar o CSV
df = pd.read_csv("consolidated_classifications.csv")

# Função para calcular métricas por grupo (modelo + fonte)
def calcular_metricas(grupo):
    y_true = grupo["true_positive"] + grupo["true_negative"] > 0  # se for verdadeiro positivo ou negativo
    y_pred = ~grupo["false_positive"].astype(bool)  # predição correta se não for falso positivo

    # Isso assume que estamos interessados na predição de INTERESTING como positivo
    tp = grupo["true_positive"].sum()
    tn = grupo["true_negative"].sum()
    fp = grupo["false_positive"].sum()
    fn = grupo["false_negative"].sum()

    total = tp + tn + fp + fn

    acc = (tp + tn) / total if total else 0
    prec = tp / (tp + fp) if (tp + fp) else 0
    rec = tp / (tp + fn) if (tp + fn) else 0
    f1 = 2 * (prec * rec) / (prec + rec) if (prec + rec) else 0

    return pd.Series({
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1_score": f1
    })

# Aplicar por modelo + fonte
metricas_por_grupo = df.groupby(["model", "font"]).apply(calcular_metricas).reset_index()

# Exibir
display(metricas_por_grupo)
metricas_por_grupo.to_csv("result_table.csv", decimal=',', sep=';', index=False)


C:\Users\lucas\AppData\Local\Temp\ipykernel_19628\3569469697.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metricas_por_grupo = df.groupby(["model", "font"]).apply(calcular_metricas).reset_index()


,model,font,accuracy,precision,recall,f1_score
0,deepseek-r1_14b,data,0.784314,0.800000,0.87500,0.835821
1,deepseek-r1_14b,gemma3_4b,0.823529,0.848485,0.87500,0.861538
2,deepseek-r1_14b,llama3.2_3b,0.901961,0.909091,0.93750,0.923077
3,deepseek-r1_14b,phi4-mini,0.823529,0.870968,0.84375,0.857143
4,deepseek-r1_14b,tinyllama_1.1b,0.843137,0.875000,0.87500,0.875000
5,gemma3_12b,data,0.745098,0.711111,1.00000,0.831169
6,gemma3_12b,gemma3_4b,0.745098,0.711111,1.00000,0.831169
7,gemma3_12b,llama3.2_3b,0.725490,0.695652,1.00000,0.820513
8,gemma3_12b,phi4-mini,0.705882,0.680851,1.00000,0.810127
9,gemma3_12b,tinyllama_1.1b,0.705882,0.680851,1.00000,0.810127


In [6]:
import pandas as pd

df = pd.read_csv("consolidated_classifications.csv")

grouped = df.groupby(["model", "font"])[
    ["false_positive", "false_negative", "true_positive", "true_negative"]
].sum().reset_index()

display(grouped)
grouped.to_csv("result_table_brute.csv", decimal=',', sep=';', index=False)

,model,font,false_positive,false_negative,true_positive,true_negative
0,deepseek-r1_14b,data,7,4,28,12
1,deepseek-r1_14b,gemma3_4b,5,4,28,14
2,deepseek-r1_14b,llama3.2_3b,3,2,30,16
3,deepseek-r1_14b,phi4-mini,4,5,27,15
4,deepseek-r1_14b,tinyllama_1.1b,4,4,28,15
5,gemma3_12b,data,13,0,32,6
6,gemma3_12b,gemma3_4b,13,0,32,6
7,gemma3_12b,llama3.2_3b,14,0,32,5
8,gemma3_12b,phi4-mini,15,0,32,4
9,gemma3_12b,tinyllama_1.1b,15,0,32,4
